<a href="https://colab.research.google.com/github/20pt32/KLA_Hackaton/blob/main/Milestone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import re
from rtree import index

def find_similar_polygons(poi_file, source_file, output_file):
    # function to extract polygon points from a string
    def extract_points(polygon_str):
        pattern = r"xy\s+\d+\s+(.*)endel"
        match = re.search(pattern, polygon_str)
        if match:
            points = match.group(1).split()
            print("Matched: ", match.group(0))
            print("Points: ", points)
            return [ (float(points[i]), float(points[i+1])) for i in range(0, len(points), 2) ]
        print("No Match")
    return None


    # create an R-tree index
    idx = index.Index()

    # read the POI polygon
    with open(poi_file, "r") as f:
        poi_str = f.read()
        poi_points = extract_points(poi_str)
        minx = min(point[0] for point in poi_points)
        miny = min(point[1] for point in poi_points)
        maxx = max(point[0] for point in poi_points)
        maxy = max(point[1] for point in poi_points)
        bounds = (minx, miny, maxx, maxy)
        idx.insert(0, bounds, poi_points)


    # open the output file
    with open(output_file, "w") as out_f:
        # read the source file and find similar polygons
        with open(source_file, "r") as src_f:
            for line in src_f:
                if "xy" in line:
                    poly_str = line
                    while "endel" not in line:
                        line = next(src_f)
                        poly_str += line
                    poly_points = extract_points(poly_str)
                    # check if the polygon is similar to the POI polygon
                    for i in idx.intersection(poly_points, objects=True):
                        if i.object == poi_points:
                            out_f.write(poly_str)
                            break

# call the function
find_similar_polygons("POI.txt", "Source.txt", "output.txt")
